In [1]:
!pip install numpy

In [2]:
!pip install opencv-python

In [3]:
!pip install PySimpleGUI

In [4]:
import numpy as np
import cv2
import os.path
import PySimpleGUI as sg

In [5]:
version = "7 June 2020"

prototxt = r"colorization_deploy_v2.prototxt"
model = r"colorization_release_v2.caffemodel"
points = r"pts_in_hull.npy"

points = os.path.join(os.path.dirname("./"), points)
prototxt = os.path.join(os.path.dirname("./"), prototxt)
model = os.path.join(os.path.dirname("./"), model)
if not os.path.isfile(model):
    sg.popup_scrolled(
        "Missing model file",
        'You are missing the file "colorization_release_v2.caffemodel"',
        'Download it and place into your "model" folder',
        "You can download this file from this location:\n",
        r"https://www.dropbox.com/s/dx0qvhhp5hbcx7z/colorization_release_v2.caffemodel?dl=1",
    )
    exit()
net = cv2.dnn.readNetFromCaffe(prototxt, model)  # load model from disk
pts = np.load(points)

In [6]:
# add the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [ ]:
import cv2
import numpy as np
import os
import PySimpleGUI as sg


# Function to colorize an image (either from file or frame)
def colorize_image(image_filename=None, cv2_frame=None):
    image = cv2.imread(image_filename) if image_filename else cv2_frame
    if image is None:
        sg.popup_error("Error: Unable to load image.")
        return None, None
    scaled = image.astype("float32") / 255.0
    lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)

    resized = cv2.resize(lab, (224, 224))
    L = cv2.split(resized)[0]
    L -= 50

    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

    L = cv2.split(lab)[0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)
    colorized = (255 * colorized).astype("uint8")
    return image, colorized


# Function to process a video and colorize frame-by-frame
def process_video(video_path, window, output_path=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        sg.popup_error("Error: Unable to open video file.")
        return

    fourcc = None
    if output_path:
        if output_path.lower().endswith(".avi"):
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
        elif output_path.lower().endswith(".mp4"):
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        elif output_path.lower().endswith(".webm"):
            fourcc = cv2.VideoWriter_fourcc(*"VP90")
        else:
            sg.popup_error("Unsupported file format. Use .avi, .mp4, or .webm")
            return

        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_size = (
            int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        )
        out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)
    else:
        out = None

    stop_processing = False
    while cap.isOpened() and not stop_processing:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = convert_to_grayscale(frame)
        window["-IN-"].update(data=cv2.imencode(".png", gray_frame)[1].tobytes())
        _, colorized_frame = colorize_image(cv2_frame=gray_frame)
        if colorized_frame is None:
            break
        if out:
            out.write(colorized_frame)
        colorized_frame = cv2.resize(colorized_frame, (400, 400))
        imgbytes = cv2.imencode(".png", colorized_frame)[1].tobytes()
        window["-OUT-"].update(data=imgbytes)
        event, _ = window.read(timeout=10)
        if event in (None, "Exit", "-STOP-"):
            stop_processing = True
    cap.release()
    if out:
        out.release()


# Function to convert an image to grayscale
def convert_to_grayscale(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return np.stack((gray,) * 3, axis=-1)


# Define UI theme
sg.theme("DarkBlue3")

# Define UI layout
layout = [
    [
        sg.Text("Input File:", font=("Arial", 14)),
        sg.InputText(size=(50, 1), enable_events=True, key="-IN FILE-"),
        sg.FileBrowse(button_text="Browse", size=(10, 1)),
    ],
    [
        sg.Button("Colorize Photo", key="-PHOTO-", size=(15, 1)),
        sg.Button("Colorize Video", key="-VIDEO-", size=(15, 1)),
        sg.Button("Stop", key="-STOP-", size=(10, 1)),
        sg.Button("Save Image", key="-SAVE-", size=(12, 1)),
        sg.Button("Save Video", key="-SAVE VIDEO-", size=(12, 1)),
        sg.Button("Exit", size=(10, 1)),
    ],
    [
        sg.Frame(
            "Original Image/Video",
            [[sg.Image(filename="", key="-IN-", size=(400, 400))]],
        ),
        sg.Frame(
            "Colorized Image/Video",
            [[sg.Image(filename="", key="-OUT-", size=(400, 400))]],
        ),
    ],
]

# Create window
window = sg.Window(
    "Photo & Video Colorizer", layout, size=(900, 650), resizable=True, finalize=True
)

# Event loop
while True:
    event, values = window.read()
    if event in (None, "Exit"):
        break
    elif event == "-PHOTO-":
        filename = values["-IN FILE-"]
        if filename:
            image, colorized = colorize_image(filename)
            if image is not None and colorized is not None:
                image = cv2.resize(image, (400, 400))
                colorized = cv2.resize(colorized, (400, 400))
                window["-IN-"].update(data=cv2.imencode(".png", image)[1].tobytes())
                window["-OUT-"].update(
                    data=cv2.imencode(".png", colorized)[1].tobytes()
                )
    elif event == "-VIDEO-":
        filename = values["-IN FILE-"]
        if filename:
            process_video(filename, window)
    elif event == "-SAVE-":
        filename = sg.popup_get_file(
            "Save Image", save_as=True, default_extension=".png"
        )
        if filename:
            img = window["-OUT-"].Widget.image
            if img is not None:
                cv2.imwrite(filename, img)
    elif event == "-SAVE VIDEO-":
        filename = sg.popup_get_file(
            "Save Video", save_as=True, default_extension=".avi"
        )
        if filename:
            process_video(values["-IN FILE-"], window, output_path=filename)

# Close window
window.close()

OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'webm / WebM'
